In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import scipy.linalg
import matplotlib.pyplot as plt
import linear_shrinkage
import Util
from Util import evaluate_curret, get_invest_period
from os.path import join
from generating_MTP import generate_mat as generate_MTP2
import os
import time
import sklearn.covariance
import pickle
from collections import namedtuple, defaultdict
from subprocess import Popen
from multiprocessing import Pool

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
util_loaded_data = Util.load_data()
ret, ret_nonan, univ, tradeidx, dates = util_loaded_data

## This is mostly all done

In [ ]:
def save_past_information(T, N, util_loaded_data, save_KT = True):
    ret, ret_nonan, univ, tradeidx, dates = util_loaded_data
    pastRets = []
    KT_covs = []
    
    KT_time = 0
    for h in range(len(univ)):
        print(h, end = ' ')
        
        pastRet = Util.get_past_period(h, T, N, univ, tradeidx, ret)
        pastRetFileName = "pastRets/{}_{}_{}_pastRet.pkl".format(T, N, h)
        with open(pastRetFileName, 'wb') as f:
            pickle.dump(pastRet, f)
        pastRets.append(pastRet)
        
        if save_KT:
            #start = time.time()
            KT_cov = Util.kendall_cov(pastRet)
            KT_cov_file_name =  "pastRets/{}_{}_{}_KTcov.pkl".format(T, N, h)
            with open(KT_cov_file_name, 'wb') as f:
                pickle.dump(KT_cov, f)
            KT_covs.append(KT_cov)
            end = time.time()
            #print(end-start)
    
    with open("pastRets/{}_{}_pastRets.pkl".format(T,N), 'wb') as f:
        pickle.dump(pastRets, f)

    if save_KT:
        with open("pastRets/{}_{}_KTcovs.pkl".format(T,N), 'wb') as f:
            pickle.dump(KT_covs, f)

In [ ]:
for T in [25, 50, 100, 200, 500, 1260]:
    for N in [25, 50, 100, 200, 500, 1000]:
        save_past_information(T, N, util_loaded_data, save_KT = False)

In [ ]:
save_past_information(1000,500, util_loaded_data, save_KT=False)

In [ ]:
#for saving KT covs in parallel
def save_KT_cov(h):
    KT_cov_file_name =  "pastRets/{}_{}_{}_KTcov.pkl".format(T, N, h)
    print(h)
    if h == 0:
        print("N={}, T={}".format(N,T))
    start = time.time()
    ret, ret_nonan, univ, tradeidx, dates = util_loaded_data
    pastRet = Util.get_past_period(h, T, N, univ, tradeidx, ret)
    KT_cov = Util.kendall_cov(pastRet)
    KT_cov_file_name =  "pastRets/{}_{}_{}_KTcov.pkl".format(T, N, h)
    with open(KT_cov_file_name, 'wb') as f:
        pickle.dump(KT_cov, f)
    end = time.time()
    print(end-start)

In [ ]:
for N, T in [(200,400)]:
    pool = Pool(8)
    pool.map(save_KT_cov,range(360))

In [ ]:
#function to consolidate all the KT_covs into an array that is pickled
for N, T in [(200,400)]:#[(100,50), (100,200), (200,100), (200,400), (500,250), (500,1000)]:
    print("N, T", N, T)
    start = time.time()
    KT_covs = []
    for h in range(360):
        fname = "pastRets/{}_{}_{}_KTcov.pkl".format(T, N, h)
        with open(fname, 'rb') as f:
            cov = pickle.load(f)
            KT_covs.append(cov)
    with open("pastRets/{}_{}_KTcovs.pkl".format(T,N), 'wb') as f:
            pickle.dump(KT_covs, f)
    print(time.time()-start)

In [ ]:
def save_outrets(N, util_loaded_data, P=1):
    ret, ret_nonan, univ, tradeidx, dates = util_loaded_data
    all_outrets = []
    for h in range(360):
        outret = get_invest_period(h, P, N, univ, tradeidx, ret)
        all_outrets.append(outret)
    with open('outRets/{}_outRets.pkl'.format(N), 'wb') as f:
        pickle.dump(all_outrets, f)

In [ ]:
for N in [25, 50, 100, 200, 500, 1000]:
    start = time.time()
    save_outrets(N, util_loaded_data)

## After this point is the algorithms and experiments, above was just generating data

In [4]:
from estimators import (MTP2_wrapper, MTP2_cov_wrapper,
                        LRPS_wrapper,
                        CLIME_wrapper, CLIME_cov_wrapper,
                        old_LS_wrapper, LS_wrapper, NLS_wrapper,
                        POET_wrapper, POET_5_wrapper,
                        glasso_wrapper,
                        get_AFM_estimator)

In [14]:
all_methods = { #all time estimates for N = 100
    'glasso': glasso_wrapper, #2 seconds
    'MTP2': MTP2_wrapper, #25 seconds
    #'MTP2_cov': MTP2_cov_wrapper,
    'CLIME': CLIME_wrapper,
    #'CLIME_cov': CLIME_cov_wrapper
    #'LS': LS_wrapper, #0.5 seconds
    #'old_LS_cov': old_LS_cov_wrapper,
    'old_LS': old_LS_wrapper,
    'NLS':  NLS_wrapper, #5 seconds
    'LRPS': LRPS_wrapper, #7 seconds
    'AFM_NL': get_AFM_estimator(5, 'NLS', tradeidx), #4 seconds
    'AFM_LS': get_AFM_estimator(5, 'LS', tradeidx), #0.5 seconds
    'POET': POET_wrapper,
    'POET_5': POET_5_wrapper,
    #'AFM_POET': None,
    #'equiweight': None,
    #'POET': POET_wrapper
}

### Time estimate

In [15]:
T = 1260
N = 100
with open('pastRets/{}_{}_pastRets.pkl'.format(T,N), 'rb') as f:
    pastRets = pickle.load(f)
timing_dict = {}
for method_name, m_func in all_methods.items():
    h = 0
    print(method_name, end = ' ')
    args = []
    start = time.time()
    if 'AFM' in method_name:
        args = [h]
    cov = m_func(pastRets[h], *args)
    end = time.time()
    print(end-start)
    timing_dict[method_name] = end-start

glasso 

/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


2.037687063217163
MTP2 fd78b188-d8d9-4d9b-a387-866878a11eec
16.953660011291504
CLIME b'[1] "Loaded X"\n'
b'[1] 0.03984095\n'
b''
6.4462409019470215
old_LS 0.10021209716796875
NLS b'[1] "Loaded X"\n'
b'Estimating population eigenvalues...[1] "Finished with NLS"\n'
b''
3.3802690505981445
LRPS b'[1] "### Computing the path on the full dataset first ###"\n'
b'[1] "Fitting with gamma= 0.15  and lambda= 1.13453970662616 Sparsity: 0.00484848484848485 Rank of L: 3"\n'
b'[1] "Fitting with gamma= 0.15  and lambda= 0.568616817004261 Sparsity: 0.0121212121212121 Rank of L: 6"\n'
b'[1] "Fitting with gamma= 0.15  and lambda= 0.284983489508311 Sparsity: 0.042020202020202 Rank of L: 15"\n'
b'[1] "Fitting with gamma= 0.15  and lambda= 0.142830086736117 Sparsity: 0.145252525252525 Rank of L: 29"\n'
b'[1] "Fitting with gamma= 0.15  and lambda= 0.0715846160500178 Sparsity: 0.307878787878788 Rank of L: 36"\n'
b'[1] "Fitting with gamma= 0.15  and lambda= 0.0358772956883789 Sparsity: 0.430909090909091 Rank o

In [18]:
print(timing_dict.items())
print("Total hours: ", sum(timing_dict.values()))

dict_items([('glasso', 2.037687063217163), ('MTP2', 16.953660011291504), ('CLIME', 6.4462409019470215), ('old_LS', 0.10021209716796875), ('NLS', 3.3802690505981445), ('LRPS', 14.036257982254028), ('AFM_NL', 1.3425300121307373), ('AFM_LS', 0.15319514274597168), ('POET', 18.358459949493408), ('POET_5', 18.198095321655273)])
Total hours:  81.00660753250122


In [19]:
times = []
for h in range(20):
    start = time.time()
    cov = glasso_wrapper(pastRets[0])
    end = time.time()
    times.append(end-start)

/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The defaul

In [21]:
print(times)
print(sum(times))
print(sum(times)/len(times))

[2.059591054916382, 1.8806960582733154, 1.8820281028747559, 1.93168306350708, 1.9128968715667725, 2.039340019226074, 2.06835675239563, 2.0088319778442383, 2.000925064086914, 2.0052478313446045, 2.022662878036499, 3.0579960346221924, 3.531808614730835, 2.0662379264831543, 1.9228878021240234, 2.24661922454834, 3.633918285369873, 2.306779146194458, 1.894467830657959, 2.046090841293335]
44.519065380096436
2.2259532690048216


### Run versions

In [10]:
run_name = "test_run_new"
KT_or_not = False

for N, T in [(100,1260)]:

    with open("run_info/{}.info".format(run_name), 'wb') as f:
        run_info = {
            'T': T,
            'N': N,
            'KT_or_not': KT_or_not,
            'all_methods': None#all_methods
        }
        pickle.dump(run_info, f)

    with open('pastRets/{}_{}_pastRets.pkl'.format(T,N), 'rb') as f:
        pastRets = pickle.load(f)

    if os.path.isfile('pastRets/{}_{}_KTcovs.pkl'.format(T,N)):
        with open('pastRets/{}_{}_KTcovs.pkl'.format(T,N), 'rb') as f:
            pastCovs = pickle.load(f)
    else:
        print("pastCovs doesn't exist for T={} N={}".format(T,N))
        pastCovs = []

    with open('outRets/{}_outRets.pkl'.format(N), 'rb') as f:
        outRets = pickle.load(f)

    #assert len(pastRets) == 360
    #assert len(outRets) == 360

    print("Loaded all relevant information")

    def get_covs_all_methods(h):
        print("WORKING ON h={}".format(h))
        for method_name, method in all_methods.items():
            print('Starting on {}'.format(method_name))

            #USING PASTCOVS
            if 'cov' in method_name:
                args = []
                if 'LS' in method_name or 'CLIME' in method_name:
                    #args is number of samples
                    args = [T]
                cov = method(cov=pastCovs[h], *args)
            else:
            #USING PASTRETS
                args = []
                if 'AFM' in method_name:
                    args = [h]
                cov = method(pastRets[h], *args)

            #print(cov)
            with open('pickle/{}_{}_{}_{}_{}_{}_covEst.pkl'.format(T, N, h, method_name, KT_or_not, run_name), 'wb') as f:
                pickle.dump(cov, f)
    
    pool = Pool(8)
    pool.map(get_covs_all_methods, range(16))

Loaded all relevant information
WORKING ON h=1
WORKING ON h=6
WORKING ON h=5
WORKING ON h=0
WORKING ON h=3
WORKING ON h=4
WORKING ON h=2
WORKING ON h=7
Starting on CLIME
Starting on CLIME
Starting on CLIME
Starting on CLIME
Starting on CLIME
Starting on CLIME
Starting on CLIME
Starting on CLIME
3e2d9ae0-6e3b-4933-b6b2-93e4d462128f
b1bc8aa3-ba5c-44e6-aeae-049c8d2e8682
b583d957-c37f-45e9-b22e-dd8b5013f952
cb8c8674-4edf-4abe-b5dc-4fa1cb71aaf9
1b8412d0-e4c2-47a8-9b82-1adc78cb43aa
2acf72f6-cace-4ef1-8be3-8a9b4962e0e9
0b53bdbe-34cd-4dd2-a18b-5043feb807b2
6bb06525-610a-487f-862a-d46d4f4aea31
b'[1] "Loaded X"\n'
b'[1] 0.03984095\n'
b'[1] "Loaded X"\n'
b'[1] 0.03984095\n'
b'[1] "Loaded X"\n'
b'[1] "Loaded X"\n'
b'[1] "Loaded X"\n'
b'[1] 0.03984095\n'
b'[1] 0.03984095\n'
b'[1] "Loaded X"\n'
b'[1] 0.03984095\n'
b'[1] "Loaded X"\n'
b'[1] 0.03984095\n'
b'[1] 0.03984095\n'
b'[1] "Loaded X"\n'
b'[1] 0.03984095\n'
b''
Starting on NLS
b''
b''
b''
Starting on NLS
b''
b''
Starting on NLS
Starting on NLS


Process ForkPoolWorker-7:
Process ForkPoolWorker-8:
Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Process ForkPoolWorker-6:
Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Process ForkPoolWorker-5:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.fram